# Rebalanceamento de Carteira:

### Import de Bibliotecas:

In [1]:
import re
import locale
import numpy as np
import pandas as pd
from skimage.measure import label, regionprops

locale.setlocale(locale.LC_ALL, 'pt-BR.utf-8')

'pt-BR.utf-8'

### Leitura dos Excels:

In [2]:
path = './Dados'

In [3]:
carteira = pd.read_excel(f'{path}/Portfolio Global - KING.xlsx', sheet_name='Carteira Global', header=None)
politica = pd.read_excel(f'{path}/Portfolio Global - KING.xlsx', sheet_name='Política de Investimento', header=None)
database = pd.read_excel(f'{path}/Planilha PFE Classificadora.xlsx', sheet_name='result', header=None)

### Extração de Tabelas e Atribuição de Variáveis Dataframe

#### Análise de Tabelas dentro de cada Excel:

In [4]:
def getTables(df):
    larr = label(np.array(df.notnull()).astype("int"))
    
    list_dfs = []
    
    for s in regionprops(larr):
        sub_df = df.iloc[
            s.bbox[0]:s.bbox[2], s.bbox[1]:s.bbox[3]
        ].pipe(
            lambda df_: df_.rename(columns=df_.iloc[0]).drop(df_.index[0])
        )
    
        list_dfs.append(sub_df)
    return list_dfs


In [5]:
dfsCarteira = getTables(carteira)
dfsPolitica = getTables(politica)
dfsDatabase = getTables(database)

#### Criando Variáveis para cada Tabela dos Excels

In [6]:
def attrDataframes(list_dfs):
    def getVarName(df):
        def namestr(obj, namespace):
            return [name for name in namespace if namespace[name] is obj]
    
        return namestr(df, globals())[0]
    
    for i in range(len(list_dfs)):
        dfName = getVarName(list_dfs).replace('dfs', '')
        
        varName = f'df{dfName}{i+1}'
        value = list_dfs[i]
        globals()[varName] = value
        
        print(f'- Dataframe {varName} created.')
    print('')


In [7]:
attrDataframes(dfsCarteira)
attrDataframes(dfsPolitica)
attrDataframes(dfsDatabase)

- Dataframe dfCarteira1 created.

- Dataframe dfPolitica1 created.
- Dataframe dfPolitica2 created.
- Dataframe dfPolitica3 created.

- Dataframe dfDatabase1 created.



### Dataframes:

#### - Carteira:

In [8]:
valorCarteira = dfCarteira1["Financeiro"].iloc[-1]
perctCarteira = dfCarteira1["%"].iloc[-1]

In [9]:
dfCarteira1 = dfCarteira1.dropna(subset=["Source"])
dfCarteira1 = dfCarteira1.dropna(subset=["Portfólio"])

In [10]:
dfCarteira1.head()

,Data Posição,Source,Portfólio,Quantidade,Financeiro,Classe,%
5,2023-09-13 00:00:00,KING - BRL,BV CASH FIRF SIMPLES,21170.299483,26943.031989,Renda Fixa Brasil Pós Fixado,0.002191
6,2023-09-13 00:00:00,KING - BRL,BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF,218395.712765,315562.17117,Renda Fixa Brasil Pós Fixado,0.025664
7,2023-09-13 00:00:00,KING - BRL,LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO,800000,820257.249879,Renda Fixa Brasil Pós Fixado,0.06671
8,2023-09-13 00:00:00,KING - BRL,LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO,1000000,1004178.335827,Renda Fixa Brasil Pós Fixado,0.081668
9,2023-09-13 00:00:00,KING KONG FIM CP IE,LFT01092026,38,264782.350653,Renda Fixa Brasil Pós Fixado,0.021534


#### - Política:

In [11]:
dfPolitica1.head()

,TradingDesk,Book,Min,Max,Estratégico,Tático,Atual
1,KING - BRL,Renda Fixa BR Pós-Fixado,0.05,0.35,0.075,0.13,0.233557
2,KING - BRL,Renda Fixa BR Crédito Privado,0,0.2,0.09,0.09,0.100853
3,KING - BRL,Renda Fixa BR Pré-Fixado,0,0.1,0.04,0,0
4,KING - BRL,Renda Fixa BR Inflação,0,0.15,0.08,0.12,0.109807
5,KING - BRL,Renda Fixa Intl Cash Equivalent,0,0.05,0,0.005,0.004163


In [12]:
dfPolitica2.rename(columns={'%':'% Target'}, inplace=True)
dfPolitica2.head()

,Alocação Estratégica por Mercado,% Target
23,Renda Fixa,0.315
24,Retorno Absoluto,0.15
25,Renda Variável,0.225
26,Private Equity,0.255
27,Real Estate,0.055


In [13]:
dfPolitica3.head()

,Alocação Estratégica por Geografia,%
31,Brasil,0.575
32,Internacional,0.425


#### - Database:

In [14]:
dfDatabase1.head()

,AssetClassName,AssetClassLevelTwoName,Benchmark,AssetClassLevelThreeName,AssetName,AssetInvestmentTypeName,AssetRedemptionGate,AssetRedemptionLockUpDayCountName,AssetRedemptionLockUpDayCountValue,AssetRedemptionMonths,AssetRedemptionQuotationDelayDayCountName,AssetRedemptionQuotationDelayDayCountValue,AssetRedemptionQuotationFrequencyName,AssetRedemptionSettlementDelayDayCountName,AssetRedemptionSettlementDelayDayCountValue,AssetIssueMonths,AssetIssueDelayDayCountName,AssetIssueDelayDayCountValue,AssetIssueQuotationFrequencyName
2,Private Equity,Private Equity Internacional,S&P500,Private Equity Internacional Multiestratégia,Jera - JWM Illiquid SP III- I-U- Initial,InvestmentFundOffShore,100.0,Meses,60.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,30.0,Primeiro dia do Mês,Dias Corridos,30.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,0,Primeiro dia do Mês
3,Caixa / Conta Corrente / Provisões,Caixa / Conta Corrente / Provisões,CDI,Caixa / Conta Corrente / Provisões,Empréstimo USD Tapioca,Provisions and Costs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Private Equity,Private Equity Internacional,S&P500,Private Equity Internacional Venture Capital,Coller International Partners VIII Feeder Fund,InvestmentFundOffShore,100.0,Dias Corridos,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia,Dias Úteis,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia
5,Renda Fixa Brasil Crédito Pós-Fixado,RF BR Crédito Pós-Fixado Crédito Estruturado,CDI,NaN,JWM Crédito FIM CP,InvestmentFund,100.0,Dias Corridos,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,180.0,Fechamento do Dia,Dias Corridos,1.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia
6,Renda Fixa,Renda Fixa Brasil Inflação,CDI,RF BR Inflação Soberano,JWM Inflação FIM CP,InvestmentFund,100.0,Dias Corridos,0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Corridos,7.0,Fechamento do Dia,Dias Úteis,1.0,[ 1 2 3 4 5 6 7 8 9 10 11 12],Dias Úteis,0,Fechamento do Dia


### Rebalanceamento

#### Pegando Ativos da Carteira

In [15]:
result_list = dfCarteira1['Portfólio'].dropna().to_list()

result_list

['BV CASH FIRF SIMPLES',
 'BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF',
 'LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO',
 'LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO',
 'LFT01092026',
 'TESOURO SELIC FI RF',
 'JWM CREDITO FIM CP',
 'JWM INFLAÇÃO FIM CP',
 'JWM MULTIMERCADO FIM IE',
 'BOVA11',
 'JWM AÇÕES FIA',
 'MAESTRO FIA',
 'IVVB11',
 'NOVA RAPOSO FIP',
 'JWM VANQUISH FIM CP IE',
 'SPECTRA NIMBUS FIP',
 'JWM ILLIQUID SP III',
 'GTWR11 - FII G TOWERS CI',
 'FII VOTORANTIM LECREC',
 'JWM LIQUID SP II',
 'WDOV23']

#### Categorizando cada Ativo com seu Benchmark

In [16]:
cont = 0
benchmarks = []

for i in result_list:
    source = dfDatabase1['AssetName'].astype(str).str.lower()
    search = i.lower()

    source = source.apply(lambda x: re.sub(r'\d+', '', x))
    search = re.sub(r'\d+', '', search)
    
    result = dfDatabase1[source == search][['Benchmark', 'AssetName']]
    
    if not result.empty:
        cont += 1
        benchmarks.append(result['Benchmark'].iloc[0])
        print(result['Benchmark'].iloc[0])
    else:
        benchmarks.append(np.nan)
        print("NÃO PASSEI", end=' | ')
        print(i)

    # print(source)
    print(search)
    print("")

    

print(f"{cont/len(result_list):.2%}")

NÃO PASSEI | BV CASH FIRF SIMPLES
bv cash firf simples

CDI
btg pactual digital tesouro selic simples fi rf

NÃO PASSEI | LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO
lci jun - jun . bradesco

NÃO PASSEI | LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO
lca aug - aug . bradesco

NÃO PASSEI | LFT01092026
lft

NÃO PASSEI | TESOURO SELIC FI RF
tesouro selic fi rf

NÃO PASSEI | JWM CREDITO FIM CP
jwm credito fim cp

CDI
jwm inflação fim cp

IBOVESPA
jwm multimercado fim ie

IBOVESPA
bova

IBOVESPA
jwm ações fia

NÃO PASSEI | MAESTRO FIA
maestro fia

S&P500
ivvb

NÃO PASSEI | NOVA RAPOSO FIP
nova raposo fip

IBOVESPA
jwm vanquish fim cp ie

IBOVESPA
spectra nimbus fip

NÃO PASSEI | JWM ILLIQUID SP III
jwm illiquid sp iii

NÃO PASSEI | GTWR11 - FII G TOWERS CI
gtwr - fii g towers ci

NÃO PASSEI | FII VOTORANTIM LECREC
fii votorantim lecrec

NÃO PASSEI | JWM LIQUID SP II
jwm liquid sp ii

NÃO PASSEI | WDOV23
wdov

38.10%


In [17]:
try:
    dfCarteira1.insert(2, 'Benchmarks', benchmarks)
except:
    pass

dfCarteira1.head()

,Data Posição,Source,Benchmarks,Portfólio,Quantidade,Financeiro,Classe,%
5,2023-09-13 00:00:00,KING - BRL,NaN,BV CASH FIRF SIMPLES,21170.299483,26943.031989,Renda Fixa Brasil Pós Fixado,0.002191
6,2023-09-13 00:00:00,KING - BRL,CDI,BTG PACTUAL DIGITAL TESOURO SELIC SIMPLES FI RF,218395.712765,315562.17117,Renda Fixa Brasil Pós Fixado,0.025664
7,2023-09-13 00:00:00,KING - BRL,NaN,LCI 30Jun2023 - 20Jun2025 96.5 BRADESCO,800000,820257.249879,Renda Fixa Brasil Pós Fixado,0.06671
8,2023-09-13 00:00:00,KING - BRL,NaN,LCA 30Aug2023 - 19Aug2025 94.5 BRADESCO,1000000,1004178.335827,Renda Fixa Brasil Pós Fixado,0.081668
9,2023-09-13 00:00:00,KING KONG FIM CP IE,NaN,LFT01092026,38,264782.350653,Renda Fixa Brasil Pós Fixado,0.021534


#### Agrupando os Benchmarks

In [18]:
groupedBenchmarks = dfCarteira1.groupby('Benchmarks')['%'].sum().reset_index()
groupedBenchmarks = groupedBenchmarks.rename(columns={'%':'% Atual'})

groupedBenchmarks

,Benchmarks,% Atual
0,CDI,0.135471
1,IBOVESPA,0.296089
2,S&P500,0.040759


#### Comparando Atual x Target

In [19]:
dicionario = {
    'CDI':'Renda Fixa',
    'IBOVESPA':'Retorno Absoluto',
    'S&P500':'Renda Variável',
    'IFIX':'Private Equity',
    'IFIX':'Real Estate',
    'ICB':'Commodities'
}

groupedBenchmarks['Benchmarks'] = groupedBenchmarks['Benchmarks'].map(dicionario)

groupedBenchmarks

,Benchmarks,% Atual
0,Renda Fixa,0.135471
1,Retorno Absoluto,0.296089
2,Renda Variável,0.040759


In [20]:
dfRebalanced = dfPolitica2.merge(groupedBenchmarks, how='left', left_on='Alocação Estratégica por Mercado', right_on='Benchmarks')
dfRebalanced['% Atual'].fillna(0, inplace=True)
dfRebalanced.drop(columns=['Benchmarks'], inplace=True)
dfRebalanced['Offset'] = dfRebalanced['% Target'] - dfRebalanced['% Atual']

dfRebalanced

,Alocação Estratégica por Mercado,% Target,% Atual,Offset
0,Renda Fixa,0.315,0.135471,0.179529
1,Retorno Absoluto,0.15,0.296089,-0.146089
2,Renda Variável,0.225,0.040759,0.184241
3,Private Equity,0.255,0.000000,0.255
4,Real Estate,0.055,0.000000,0.055
5,Commodities,0,0.000000,0.0


#### Tracking Error:

In [21]:
# ADAPTAR PARA OS FUNDOS

import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

def get_returns(ticker, period):
    start_date = (datetime.today() - timedelta(days=period)).strftime('%Y-%m-%d')
    end_date = datetime.today().strftime('%Y-%m-%d')
    data = yf.download(ticker, start=start_date, end=end_date)

    data['Return'] = data['Adj Close'].pct_change()

    return data['Return']


def calculateTrackingError(stock, benchmark='^GSPC', period=30):
    stockReturns = get_returns(stock, period)
    benchmarkReturns = get_returns(benchmark, period)

    excessReturns = stockReturns - benchmarkReturns

    trackingError = np.std(excessReturns.dropna())

    return trackingError


stock = 'AAPL'
# benchmark = '^GSPC'
period = 30

te = calculateTrackingError(stock, period=period)
print(f"The Tracking Error is {te:.2%}")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
The Tracking Error is 0.78%


In [22]:
dfRebalanced['Need Rebalance'] = [True, True, False, True, False, False]

dfRebalanced

,Alocação Estratégica por Mercado,% Target,% Atual,Offset,Need Rebalance
0,Renda Fixa,0.315,0.135471,0.179529,True
1,Retorno Absoluto,0.15,0.296089,-0.146089,True
2,Renda Variável,0.225,0.040759,0.184241,False
3,Private Equity,0.255,0.000000,0.255,True
4,Real Estate,0.055,0.000000,0.055,False
5,Commodities,0,0.000000,0.0,False


### Resultado Final:

In [23]:
if dfRebalanced['Need Rebalance'].any():
    pValue = locale.format_string("%.3f", valorCarteira, grouping=True, monetary=True)
    print(f"Portfolio Value: {pValue}\n")
    
    for i in range(len(dfRebalanced.columns) + 1):
        if dfRebalanced['Offset'][i] < 0:
            acao = 'Buy' 
        elif dfRebalanced['Offset'][i] >  0:
            acao = 'Sell'
        else:
            acao = 'Skip'

        price = abs(valorCarteira * dfRebalanced['Offset'][i])
        price = locale.format_string("%.3f", price, grouping=True, monetary=True)
        print(f"[{dfRebalanced['Alocação Estratégica por Mercado'][i]}]\n- {acao} {price} of Porfolio Value ({abs(dfRebalanced['Offset'][i]):,.3%})\n\n")

Portfolio Value: 12.295.848,806

[Renda Fixa]
- Sell 2.207.464,755 of Porfolio Value (17.953%)


[Retorno Absoluto]
- Buy 1.796.291,573 of Porfolio Value (14.609%)


[Renda Variável]
- Sell 2.265.399,696 of Porfolio Value (18.424%)


[Private Equity]
- Sell 3.135.441,446 of Porfolio Value (25.500%)


[Real Estate]
- Sell 676.271,684 of Porfolio Value (5.500%)


[Commodities]
- Skip 0,000 of Porfolio Value (0.000%)


